<h1> A380 </h1>



Des passagers embarquent à bord d'un Airbus A380. L'avion est complet. Idéalement,
les passagers devraient embarquer un à un et s'asseoir à leur place. Cependant, le premier
passager, étourdi, s'assoit à une place aléatoire. Ensuite, les passagers s'assoient à leur place
si elle est libre, et à une place libre, aléatoirement, sinon. Vous embarquez le dernier. Quelle
est la probabilité que votre place soit occupée ?

<ol>
<li>Ecrire une fonction <tt>place_du_dernier_libre nbre_places</tt> qui prend en argument <tt>nbre_places</tt> le nombre total de place dans l'avion <i>(ex: pour un A380 en configuration standard: 525 places)</i> et qui renvoie un boolean indiquant si oui ou non la derniere place est libre.
<li>Ecrire une fonction <tt>probabilite nbre_places nbre_tests</tt> qui appelle <tt>(place_du_dernier_libre nbre_places)</tt> 
<tt>nbre_tests</tt> fois et qui renvoie une estimation de la probabilité que votre place soit occupée.
</ol>

In [1]:
open Random;;
Random.self_init;;

let range debut fin =
   let rec range i acc =
     if i=fin then List.rev acc
     else range (i+1) (i::acc) in
   range debut [];; 
   
let shuffle d =
    let nd = List.map (fun c -> (Random.bits (), c)) d in
    let sond = List.sort compare nd in
    List.map snd sond;;

- : unit -> unit = <fun>

val range : int -> int -> int list = <fun>

val shuffle : 'a list -> 'a list = <fun>

In [2]:
let place_du_dernier_libre nbre_places =
    let nbre_pasagers=nbre_places in
    let liste_places_libres=Array.make nbre_places 0 in (* places libres, numeros entre 0 et 524 *)
    
    let liste_passagers = shuffle (range 0 nbre_pasagers) in (* les passagers entrent dans le désordre *)
    
    (* let numero_premier_passager = List.hd liste_passagers in *) (* on prend le premier passager *)
    let place_premier_passager = Random.int nbre_places in (* il se place au hasard *)
    liste_places_libres.(place_premier_passager)<-1; (* sa place est maintenant occupée *)
    
    let rec remplissage liste_passagers =
    match liste_passagers with
    | [] -> raise (Failure "Empty List")
    | dernier::[] -> dernier (* on place tous les passagers sauf le premier et le dernier *)
    | place_attribuee::tl -> if liste_places_libres.(place_attribuee)=0 then (* si la place est libre *)
                                liste_places_libres.(place_attribuee)<-1 (* il l'occupe *)
                             else begin
                                let position = ref (Random.int nbre_places) in (* entre 0 et 524 inclus *)
                                while liste_places_libres.(!position)=1 do
                                    position :=Random.int nbre_places
                                done;
                                liste_places_libres.(!position)<-1
                            end; 
                            remplissage tl in
                            
    let place_attribuee = remplissage (List.tl liste_passagers) in (* dernier entré *)
    liste_places_libres.(place_attribuee)=0;;

val place_du_dernier_libre : int -> bool = <fun>

In [4]:
let probabilite nbre_places nbre_tests =
  let rec loop num count =
     if num = nbre_tests then float_of_int count /. (float_of_int nbre_tests) else
     if place_du_dernier_libre nbre_places then loop (num+1) (count+1) else loop (num+1) count in
  loop 0 0;;

val probabilite : int -> int -> float = <fun>

In [5]:
probabilite 525 10000;;

- : float = 0.4868

<h1> Démonstration </h1>

Dans la réalité, les passagers d'un avion découvrant quelqu'un à leur place ne s'effacent
pas si facilement, mais ont plutôt tendance à chasser (poliment) le passager égaré. Qu'est-ce que ce modèle plus réaliste change au problème ?

En fait, pas grand chose : les sièges occupés à un instant donné avec l'un ou l'autre modèle sont les mêmes. La seule diférence est que dans ce nouvau modèle, les passagers $P_2$ ;... ;$P_k$ sont assis à leur place et que c'est $P_1$ qui se déplace, à chaque fois qu'il est chassé d'un siège par son propriétaire légitime, jusqu'à atterrir sur $S_1$ ou $S_n$. Or, à chaque fois que $P_1$ choisit aléatoirement un siège, la probabilité de choisir $S_1$ est égale à celle de choisir Sn. Au moment où j'entrerai dans l'avion, $P_1$ sera donc sur l'un de ces deux sièges avec la même probabilité, ce qui entraîne bien que la probabilité que je trouve mon siège libre est 1/2.